In [1]:
import pandas as pd

In [2]:
df_bank = pd.read_csv("../data/bank-full.csv", delimiter=";")

In [3]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_bank, test_size=0.25, random_state=123)

In [4]:
train_df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
26999,32,unemployed,single,secondary,no,2706,no,no,cellular,21,nov,462,3,-1,0,unknown,no
16168,37,admin.,married,secondary,no,1396,yes,no,cellular,22,jul,199,2,-1,0,unknown,no
12338,22,blue-collar,married,secondary,no,-295,yes,no,unknown,26,jun,150,2,-1,0,unknown,no
6074,36,blue-collar,married,secondary,no,-870,yes,no,unknown,26,may,102,2,-1,0,unknown,no
7385,50,admin.,married,primary,no,429,no,no,unknown,29,may,60,2,-1,0,unknown,no


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33908 entries, 26999 to 15725
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        33908 non-null  int64 
 1   job        33908 non-null  object
 2   marital    33908 non-null  object
 3   education  33908 non-null  object
 4   default    33908 non-null  object
 5   balance    33908 non-null  int64 
 6   housing    33908 non-null  object
 7   loan       33908 non-null  object
 8   contact    33908 non-null  object
 9   day        33908 non-null  int64 
 10  month      33908 non-null  object
 11  duration   33908 non-null  int64 
 12  campaign   33908 non-null  int64 
 13  pdays      33908 non-null  int64 
 14  previous   33908 non-null  int64 
 15  poutcome   33908 non-null  object
 16  y          33908 non-null  object
dtypes: int64(7), object(10)
memory usage: 4.7+ MB


In [6]:
# Distribution of the target

In [7]:
train_df["y"].value_counts(normalize=True)

y
no     0.882358
yes    0.117642
Name: proportion, dtype: float64

In [8]:
# Distributions of categorical and numeric features

In [9]:
categorical_cols = list(train_df.drop(columns=["y"]).select_dtypes(include=["object"]).columns)
numerical_cols = list(train_df.select_dtypes(include=["int64"]).columns)

In [10]:
import altair as alt
alt.data_transformers.enable("vegafusion")

alt.Chart(train_df).mark_bar().encode(
    x="count()",
    y=alt.Y(alt.repeat()).type("nominal")
).repeat(
    categorical_cols, columns=3
)

alt.RepeatChart(...)

In [11]:
alt.Chart(train_df).mark_bar().encode(
    x=alt.X(alt.repeat()).type("quantitative").bin(maxbins=40),
    y="count()"
).repeat(
    numerical_cols, columns=3
)

alt.RepeatChart(...)

In [12]:
# Correlations between numeric features

In [13]:
# measure linear relationship
person_corr_df = train_df[numerical_cols].corr("pearson").unstack().reset_index()
person_corr_df.columns = ["num_variable_0", "num_variable_1", "correlation"]

corr_heatmap = alt.Chart(person_corr_df).mark_rect().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    color="correlation:Q"
).properties(
    width=250,
    height=250
)

text = alt.Chart(person_corr_df, title="Pearson Correlation").mark_text().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    text=alt.Text("correlation:Q", format=".2f")
)

corr_heatmap + text

alt.LayerChart(...)

In [14]:
# measure monotonic (incl. non-linear) relationship
spearman_corr_df = train_df[numerical_cols].corr("spearman").unstack().reset_index()
spearman_corr_df.columns = ["num_variable_0", "num_variable_1", "correlation"]

corr_heatmap = alt.Chart(spearman_corr_df, title="Spearman Correlation").mark_rect().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    color="correlation:Q"
).properties(
    width=250,
    height=250
)

text = alt.Chart(spearman_corr_df).mark_text().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    text=alt.Text("correlation:Q", format=".2f")
)

corr_heatmap + text

alt.LayerChart(...)

In [15]:
# checking for correlation between pdays and previous
# as indicated in the chart, there is no linear relationship here
pdays_prev = alt.Chart(train_df, title="pdays vs previous").mark_point().encode(
    x="pdays",
    y="previous"
)

pdays_prev_clamped = alt.Chart(train_df, title="pdays vs previous (previous <= 50)").mark_point().encode(
    x="pdays",
    y=alt.Y("previous").scale(domain=(0, 50), clamp=True)
)

pdays_prev | pdays_prev_clamped

alt.HConcatChart(...)

In [16]:
# Checking data with its description

In [17]:
import numpy as np
# check all 31 days of the month exist
np.sort(train_df["day"].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [18]:
# check that campaign >= 1; it is the number of contracts for this client during this campaign, including the last contract
train_df["campaign"].describe()

count    33908.000000
mean         2.760263
std          3.087922
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         58.000000
Name: campaign, dtype: float64

In [19]:
# As indicated, all points > 0. (The red line indicates the minimum.)
alt.Chart(train_df).mark_boxplot().encode(
    alt.X("campaign").scale(domain=(-5, 60), clamp=True)
).properties(height=50) + alt.Chart(train_df).mark_rule().encode(x="min(campaign)", color=alt.value("red"))

alt.LayerChart(...)

In [20]:
# check that previous >= 0; it is the number of contracts for this client before this campaign
train_df["previous"].describe()

count    33908.000000
mean         0.579568
std          2.387853
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        275.000000
Name: previous, dtype: float64

In [21]:
# As indicated, all points >= 0. (The red line indicates the minimum.)
alt.Chart(train_df).mark_boxplot().encode(
    alt.X("previous").scale(domain=(-5, 50), clamp=True)
).properties(height=50) + alt.Chart(train_df).mark_rule().encode(x="min(previous)", color=alt.value("red"))

alt.LayerChart(...)

In [22]:
# check that if previous = 0, then pdays = -1
train_df.loc[train_df["previous"] == 0, "pdays"].unique()

array([-1])

In [23]:
# check that if previous = 0, then outcome = "unknown"
train_df.loc[train_df["previous"] == 0, "poutcome"].unique()

array(['unknown'], dtype=object)

### Summary and Recommendations from EDA
- Generally, bar charts were created for categorical variables, and histograms for numerical variables to show illustration. Correlation heatmaps based on two different metrics were generated to investigate the relationships between numerical variables. A scatter plot specifically for `pdays` vs `previous` was created.
- Judging from the proportion of each class in the target, the dataset is unbalanced
- `job`, `education`, `contact` and `poutcome` contain values "unknown", but they could be kept, because our task is to find out the importance of features. There should be no harm to keep it.
- `contact` contains values "cellular" and "telephone" only. Cellular and telephone are kind of similar, we don’t think one channel would outperform another. Therefore, we might expect the result to tell us that `contact` is not important.  
- `poutcome` is the outcome of the previous marketing campaign. Most of the "unknown" is because of `previous = 0`. If the person has never been contacted for marketing before, it makes sense to say "unknown" for this field as "previous marketing" doesn't exist. But for those who have been reached out before, this `poutcome` might be informative! 
- The distributions of `pdays` and `previous` are heavily skewed. These variables are also correlated with 0.99 Spearman correlation score and 0.44 Pearson correlation score.
- However, upon visual inspection with a scatter plot, `pdays` and `previous` do not seem to be too correlated to be an issue. We can keep them both as features.
- Overall recommendations:
   - Ordinal encode `education`
   - One-hot encode categorical variables
   - Standardize numerical columns

### Discussion
 - In this study, we would like to understand which factors would affect the most for clients' subscription to the term deposit
 - positive label: `y = "yes"`; negative label: `y = "no"`
 - We do not want to miss any potential clients. Therefore, we would like to lower as much as possible the Type I error / false positive, i.e. clients being identified as subscribed to our term deposit but actually they didn't.
 - We will select a model that gives us a robust **precision**, so that the model could better explain clients' motivation to the subscription. We would put more trust to the feature importance recommended by the model.

In [24]:
X_train, y_train = train_df.drop(columns=["y"]), train_df["y"]
X_test, y_test = test_df.drop(columns=["y"]), test_df["y"]

In [25]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

categorical_feats = ["job", "marital", "default", "housing", "loan", "contact", "day", "month", "poutcome"]
ordinal_feats = ["education"]
numeric_feats = ["age", "balance", "duration", "campaign", "previous", "pdays"]

education_levels = ["unknown", "primary", "secondary", "tertiary"]

preprocessor = make_column_transformer(
    (OneHotEncoder(sparse_output=False, drop="if_binary"), categorical_feats),
    (OrdinalEncoder(categories=[education_levels], dtype=int), ordinal_feats),
    (StandardScaler(), numeric_feats)
)

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

model_pipes = {
    "Baseline": DummyClassifier(strategy="most_frequent", random_state=522),
    "DecisionTree": make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=5, random_state=522)),
    "LogisticRegression": make_pipeline(preprocessor, LogisticRegression(max_iter=2000, random_state=522)),
}

In [27]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score

mod_precision_score = make_scorer(precision_score, zero_division=0)

classification_metrics = {
    "accuracy": "accuracy",
    "precision": mod_precision_score,
    "recall": "recall", 
}
cross_val_results = {}

for name, pipe in model_pipes.items():
    cross_val_results[name] = pd.DataFrame(
        cross_validate(
            pipe, 
            X_train, 
            y_train=="yes", 
            cv=5,
            return_train_score=True, 
            scoring=classification_metrics)
    ).agg(['mean', 'std']).round(3).T


In [28]:
pd.concat(
    cross_val_results,
    axis='columns'
).xs(
    'mean',
    axis='columns',
    level=1
).style.format(
    precision=2
).background_gradient(
    axis=None
)

,Baseline,DecisionTree,LogisticRegression
fit_time,0.00,0.10,0.25
score_time,0.00,0.01,0.01
test_accuracy,0.88,0.90,0.90
train_accuracy,0.88,0.90,0.90
test_precision,0.00,0.64,0.65
train_precision,0.00,0.67,0.66
test_recall,0.00,0.35,0.35
train_recall,0.00,0.37,0.36


### Observation:
 - `LogisticRegression` has slightly better test precision than the `DecisionTreeClassifier`, but the `LogisticRegression` has a smaller gap between train scores and test scores, so `LogisticRegression` is more likely to generalize.
 - Limitation: there is still room of improvements on the precision
   - In the future, if we ever need to make prediction of the subscription, we could increase the threshold to yield a better precision. So, I guess it's not so much worry here.
   - If we really keen on improving the precision on the model level, RandomForest could be a choice

In [29]:
lr_pipe = model_pipes['LogisticRegression']

lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse_output=False),
                                                  ['job', 'marital', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'day', 'month',
                                                   'poutcome']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['unknown',
                                                                              'primary',
                                                                              'secondary',
                                                                              'tertiary']],
                                                                 dtype=<class 'int'>),
                                                  ['education']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['age', 'balance', 'duration',
                                                   'campaign', 'previous',
                                                   'pdays'])])),
                ('logisticregression',
                 LogisticRegression(max_iter=2000, random_state=522))])

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_pipe.predict(X_test)))

              precision    recall  f1-score   support

          no       0.92      0.98      0.95     10003
         yes       0.66      0.34      0.45      1300

    accuracy                           0.90     11303
   macro avg       0.79      0.66      0.70     11303
weighted avg       0.89      0.90      0.89     11303



### Observation
 - The precision test score is similar to the validation score as well as the train score. Therefore, we would believe that the feature importance conclusion drawn from this model is generalizable to the future.

In [31]:
categorical_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['onehotencoder'].get_feature_names_out().tolist()
ordinal_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['ordinalencoder'].get_feature_names_out().tolist()
numeric_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['standardscaler'].get_feature_names_out().tolist()

feature_importance = pd.DataFrame({
    'feature': categorical_cols + ordinal_cols + numeric_cols, 
    'coef': lr_pipe.named_steps['logisticregression'].coef_[0].tolist(),
    'coef_abs': map(abs, lr_pipe.named_steps['logisticregression'].coef_[0].tolist())
})

### Observation
 - According to the below dataframe, in terms of the absolute value of coefficients, `poutcome`, `month` and `duration` are the top 3 features that are highly related to whether a client would subscribe to the term deposit or not.
   - If `poutcome == "success"`, clients were already experiencing the good services by the bank, so they're more willing to subscribe new products.
   - If `month == "mar"`, we are not sure why clients tend to accept the marketing and subscribe the term deposit in March, this pattern also exists in the test set, so it's not likely an over-fitting. We thought it might be related to the financial/tax period/bonus release time in Portugal, but it seems the finance year-end in Portugal is December (it could be March in some countries e.g. China). So we still can't make sense of it yet.
   - If `duration` is longer, that means the clients were more interested in the term deposit product and were more likely to stay on the call, the salesperson had time to do more pitching, so increase the chance of successful subscription

In [32]:
feature_importance.sort_values('coef_abs', ascending=False).drop(columns=["coef_abs"]).style.format(
    precision=3
).background_gradient(
    cmap="PiYG",
    vmin=-2,
    vmax=2,
    axis=None
)

,feature,coef
66,poutcome_success,1.607
59,month_mar,1.457
56,month_jan,-1.115
71,duration,1.084
20,contact_unknown,-0.963
57,month_jul,-0.918
62,month_oct,0.844
63,month_sep,0.778
53,month_aug,-0.733
16,housing_yes,-0.658


In [33]:
feature_importance.sort_values('coef', ascending=False).drop(columns=["coef_abs"]).style.format(
    precision=3
).background_gradient(
    cmap="PiYG",
    vmin=-2,
    vmax=2,
    axis=None
)

,feature,coef
66,poutcome_success,1.607
59,month_mar,1.457
71,duration,1.084
62,month_oct,0.844
63,month_sep,0.778
54,month_dec,0.605
18,contact_cellular,0.578
47,day_27,0.566
8,job_student,0.534
30,day_10,0.458
